In [22]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from logisticClassify2 import *
from sklearn import ensemble as sk_ensemble
from sklearn import tree
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
np.random.seed(0)

# The test data
X_test = np.genfromtxt("kaggle/X_test.txt",delimiter=None)
# Data Loading
X = np.genfromtxt("kaggle/X_train.txt",delimiter=None)
Y = np.genfromtxt("kaggle/Y_train.txt",delimiter=None)

X,Y = ml.shuffleData(X,Y)
X_tr, X_va, Y_tr, Y_va = ml.splitData(X, Y, 0.75)
train_shape = X_tr.shape[0]

learner_list = []

x,y = ml.bootstrapData(X_tr, Y_tr, train_shape)
x_val,y_val = ml.bootstrapData(X_va, Y_va, train_shape)

In [25]:
#Nearest Neighbor
knn = KNeighborsRegressor(n_neighbors=8)
knn.fit(X_tr, Y_tr)
YvaHat = knn.predict(X_va)
print("AUC KNN: {}".format(roc_auc_score(Y_va, YvaHat)))
YvaHat = knn.predict(X_test)

AUC KNN: 0.7324025298387549


In [26]:
#Adabooster:
from sklearn.ensemble import AdaBoostRegressor
ada_clf = AdaBoostRegressor(base_estimator = tree.ExtraTreeClassifier(),n_estimators = 100, learning_rate = 0.5)
ada_clf.fit(X_tr, Y_tr)
print("AUC Adaboost: {}".format(roc_auc_score(Y_va, ada_clf.predict(X_va))))
ada_predictions = ada_clf.predict(X_test)
learner_list.append(ada_predictions)

AUC Adaboost: 0.6715683335502818


In [27]:
#Random Forest Classifier:
dtree = sk_ensemble.RandomForestClassifier(n_estimators = 50, max_depth = 50, min_samples_leaf = 2**3)
dtree.fit(X_tr, Y_tr)
# tree_predict = dtree.predict(X_va)
print("AUC Forrest: {}".format(roc_auc_score(Y_va, dtree.predict(Xva))))
#print("AUC Forrest: {}".format(roc_auc_score(Y_tr, dtree.predict(Xtr))))
tree_predict = dtree.predict(X_test)

AUC Forrest: 0.662832095174568


In [28]:
#Ensemble:
Ypred = []
review_list = learner_list
for each_list in review_list:
    print(len(each_list))
new_learner_list = np.array(learner_list)
print(new_learner_list)
for each_item in range(len(new_learner_list[0])):
    Ypred.append(int(np.mean(new_learner_list[:, each_item]) > 0.5))

#Outputs a row ID and a confidence in class 1:
np.savetxt('Yhat_testing.txt', np.vstack( (np.arange(len(Ypred)) , Ypred[:]) ).T, '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');

200000
200000
[[ 1.  1.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
